# Reading excel sheets into python as a matrix

In [27]:
import os
import pandas as pd
import numpy as np
import copy
from openpyxl import load_workbook
    
cwd = os.getcwd()
cwd = os.chdir('C:\Users\jojoseph\Desktop\Python Models\Python\IO')

print (os.getcwd())

C:\Users\jojoseph\Desktop\Python Models\Python\IO


In [28]:
#opening the social account matrix excel files from IMPLAN -(89 sectors summarized from study area data/study area explorer )
new_MRIO= 'new_MRIO.xlsx'# open file with social account matrices
reg_soc_acc_mat=pd.ExcelFile(new_MRIO)
print (reg_soc_acc_mat.sheet_names)

#region3=reg_soc_acc_mat.parse('region2')

[u'region1', u'region2']


### Load the aggregation matrix

In [29]:
aggregation_matrix = np.loadtxt('aggregation_matrix.txt', usecols=range(89))

print aggregation_matrix

[[ 1.  1.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]


### For this exercise we will extract the first data column (employment) from each sheet and multiply it by the aggregation matrix

##### Note there are 534 rows in the excel sheets, the is because there are 6 regions (6x89=534) so we will just take the first region in our experiment

In [30]:
#opening the required files using while loop
dict_of_df = {}
aggregated_dict = {}
for sheet_name in reg_soc_acc_mat.sheet_names:
    
    df = reg_soc_acc_mat.parse(sheet_name)  # sheet name
    #dict_of_df[sheet_name] = df.as_matrix()
    #dict_of_df[sheet_name] = copy.deepcopy(df)
    dict_of_df[sheet_name] = df.iloc[:89,2:3]

for i in dict_of_df:
    print i, dict_of_df[i]
    aggregated_dict[i] = np.matmul(aggregation_matrix, dict_of_df[i])

region2     employment
0     151260.4
1         79.5
2         94.2
3         15.6
4         27.8
5          3.2
6        208.0
7          0.0
8          0.0
9          0.0
10      1287.4
11      2117.6
12        12.2
13      1044.3
14       100.9
15         0.0
16        91.1
17         0.0
18        13.3
19       170.0
20      1507.1
21       259.1
22       214.7
23         0.0
24         0.0
25         0.0
26         0.0
27         0.0
28         0.0
29         0.0
..         ...
59       993.6
60       231.5
61       996.8
62      1115.5
63       438.7
64       376.1
65       108.3
66       189.6
67         0.0
68         0.0
69         0.0
70        37.0
71         0.0
72         0.0
73         0.0
74         0.0
75         0.0
76         0.0
77         0.0
78         0.0
79       159.4
80         0.0
81       213.5
82       652.8
83         0.0
84       149.5
85         0.0
86         0.0
87         0.0
88         0.0

[89 rows x 1 columns]
region1     employment
0     151260.4
1

### now multiply the employment by the aggregation matrix.

In [31]:
aggregated_dict

{u'region1': array([[  1.51435100e+05],
        [  4.57510000e+03],
        [  6.90000000e+01],
        [  1.18380000e+03],
        [  2.29520000e+03],
        [  2.33550000e+03],
        [  4.52130000e+03],
        [  2.22820000e+03],
        [  3.70000000e+01],
        [  0.00000000e+00],
        [  1.02570000e+03],
        [  1.49500000e+02],
        [  0.00000000e+00],
        [  8.86400000e+02]]), u'region2': array([[  1.51435100e+05],
        [  4.57510000e+03],
        [  6.90000000e+01],
        [  1.18380000e+03],
        [  2.29520000e+03],
        [  2.33550000e+03],
        [  4.52130000e+03],
        [  2.22820000e+03],
        [  3.70000000e+01],
        [  0.00000000e+00],
        [  1.02570000e+03],
        [  1.49500000e+02],
        [  0.00000000e+00],
        [  8.86400000e+02]])}